# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice

import os

import joblib, pickle

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-159386
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-159386


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_cluster = 'hyper-rf-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [4]:
compute_trgets = ws.compute_targets
for comp_name,comp_type in compute_trgets.items():
    print(comp_name,comp_type.type,comp_type.provisioning_state)

hyper-rf-cluster AmlCompute Creating
notebook159386 ComputeInstance Succeeded
automl-cluster AmlCompute Succeeded


In [5]:
%%writefile conda_dependencies.yml

dependencies:
    python=3.6.2
    scikit-learn
    pip:
        azureml-defaults

Writing conda_dependencies.yml


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform , choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--C' : choice(0.03,0.3,3,10,30),
    '--max_iter' : choice(25,50,75,100)
})


# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est =  SKLearn(
    source_directory = '.',
    compute_target = comp_trget,
    entry_script = 'train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=5,
    policy=policy,
    estimator=est
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
### YOUR CODE HERE ###
hyperdrive_submission = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb
Web View: https://ml.azure.com/runs/HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-159386/workspaces/quick-starts-ws-159386&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
AmlCompute is getting created. Consider calling wait_for_completion() first


Streaming azureml-logs/hyperdrive.txt

"<START>[2021-09-27T00:21:33.908011][API][INFO]Experiment created<END>\n""<START>[2021-09-27T00:21:34.439876][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2021-09-27T00:21:34.631251][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb
Web View: https://ml.azure.com/runs/HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-159386/workspaces/quick-st

{'runId': 'HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb',
 'target': 'hyper-rf-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-27T00:21:33.621178Z',
 'endTimeUtc': '2021-09-27T00:36:19.06843Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'fcaa837c-54ee-45af-ac30-50179da9a067',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.8213270142180095',
  'best_child_run_id': 'HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb_8',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg159386.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb
Web View: https://ml.azure.com/runs/HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-159386/workspaces/quick-starts-ws-159386&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb
Web View: https://ml.azure.com/runs/HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-159386/workspaces/quick-starts-ws-159386&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb',
 'target': 'hyper-rf-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-27T00:21:33.621178Z',
 'endTimeUtc': '2021-09-27T00:36:19.06843Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'fcaa837c-54ee-45af-ac30-50179da9a067',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.8213270142180095',
  'best_child_run_id': 'HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb_8',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg159386.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
best_run = hyperdrive_submission.get_best_run_by_primary_metric() #Best Run
best_run.get_details()
print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())

ID :  HD_fd0df665-3eb9-4251-957e-47f4cdd8a5cb_8
Metrics :  {'Regularization Strength:': 10.0, 'Max iterations:': 50, 'Accuracy': 0.8213270142180095}


In [10]:
#TODO: Save the best model
final_model = best_run.register_model(model_name = 'hyper_model',model_path = 'outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [11]:
comp_trget.delete()